In [1]:
import pyodbc
import pandas as pd
import shutil

def copy_database(original_path, temp_path):
    shutil.copyfile(original_path, temp_path)

def export_access_to_dataframes(database_path):
    # Connection string for Access database
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Establish a connection to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Get a list of all tables in the database
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]

    # Loop through the tables and load each into a DataFrame
    for table in tables:
        query = f'SELECT * FROM [{table}]'
        df = pd.read_sql(query, conn)
        globals()[f'tbl_{table}'] = df  # Create a global variable with the table name

    # Close the connection
    conn.close()

def update_access_table(database_path, table_name, df):
    # Connection string for Access database
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Establish a connection to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Clear the existing table
    cursor.execute(f'DELETE FROM [{table_name}]')
    conn.commit()

    # Insert the updated data
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        placeholders = ', '.join(['?' for _ in row])
        values = tuple(row)
        sql = f'INSERT INTO [{table_name}] ({columns}) VALUES ({placeholders})'
        cursor.execute(sql, values)
    conn.commit()

    # Close the connection
    conn.close()

# Paths
original_database_path = 'E:\\digidure\\CLERUS_v1_24072024.accdb'
temp_database_path = 'E:\\digidure\\CLERUS_temp.accdb'

# Copy the database
copy_database(original_database_path, temp_database_path)

# Export data from the copied database
export_access_to_dataframes(temp_database_path)

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_20576\3201606797.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_20576\3201606797.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_20576\3201606797.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_20576

In [2]:
# Panda settings for showing data (this is foremost done to more easily explore the data while processing it)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [10]:
filtered_only_DM = tbl_999_Dm_all_drc_match[tbl_999_Dm_all_drc_match['clerus_id'].isna() & tbl_999_Dm_all_drc_match['new_clerus_id'].isna()]


In [11]:
filtered_only_DM.describe()

,pid,jaar intrede,individual_id,clerus_id,new_clerus_id
count,29463.000000,29463.000000,29463.000000,0.0,0.0
mean,26450.933374,1916.092319,22581.498727,NaN,NaN
std,15398.598118,56.827199,5430.808287,NaN,NaN
min,1.000000,1556.000000,25.000000,NaN,NaN
25%,13029.500000,1880.000000,18183.500000,NaN,NaN
50%,26111.000000,1920.000000,22633.000000,NaN,NaN
75%,39717.500000,1961.000000,27134.000000,NaN,NaN
max,53646.000000,2008.000000,31653.000000,NaN,NaN


In [13]:
filtered_only_DM.head(40)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,individual_id,clerus_id,new_clerus_id
0,35651,1,ZH,None,Oudewater,None,(Hekendorp) Brunner; L.,hulppredikant Oirschot,None,None,1867,Lobith,None,None,1868,None,13955,NaN,NaN
1,45546,2,GE,None,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,None,13956,NaN,NaN
2,7179,3,NB,None,Breda,None,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,None,13962,NaN,NaN
3,21505,3,GR,None,Hornhuizen,None,Aa; dr. P.J.B.K. Simon van der,kandidaat,None,None,1862,Goutum,None,None,1866,None,13959,NaN,NaN
4,36887,3,NH,None,Purmerend,None,Aa; dr. P.J.B.K. Simon van der,Goutum,None,None,1886,Winterswijk,None,None,1892,None,13960,NaN,NaN
5,50146,3,GE,None,Winterswijk,None,Aa; dr. P.J.B.K. Simon van der,Purmerend,None,None,1892,Breda,None,None,1893,None,13960,NaN,NaN
6,15179,3,FR,None,Goutum,None,Aa; dr. P.J.B.K. van der,Kloosterburen,None,None,1866,Purmerend,None,None,1886,None,13961,NaN,NaN
7,7204,3,NB,None,Breda,4. Wijk Noord,Aa; dr. P.J.B.K.S. van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,None,13958,NaN,NaN
8,19779,4,GE,None,Herwijnen,None,Aa; W.P. van der,kandidaat,24,december,1989,None,None,None,None,None,13957,NaN,NaN
9,35941,5,OV,None,Overdinkel,None,Aaij; J.,Hoogkarspel-De Drieslag,25,september,1983,emeritaat,1,september,1986,None,13963,NaN,NaN


In [ ]:
# Processing the data
df = tbl_999_Dm_all_drc_match
df.sort_values(by=['clerus_id', 'role_start_year'], inplace=True)
df['role_end_year'] = df.groupby('clerus_id')['role_start_year'].shift(-1).where(df['role_end_year'].isna(), df['role_end_year'])

tbl_01_clerus_bio_death = tbl_01_clerus_bio[['clerus_id', 'death_year']]
df_joined = pd.merge(df, tbl_01_clerus_bio_death, on='clerus_id', how='left')
df_joined['role_end_year'] = df_joined.apply(lambda row: row['death_year'] if pd.isna(row['role_end_year']) and not pd.isna(row['role_start_year']) else row['role_end_year'], axis=1)
df_joined.drop(columns=['death_year'], inplace=True)

# Ensure that 'role_start_year' and 'role_end_year' are converted to integers and handle NaN values
df_joined['role_start_year'] = df_joined['role_start_year'].fillna(0).astype(int)
df_joined['role_end_year'] = df_joined['role_end_year'].fillna(0).astype(int)
df_joined['role_place_id'] = df_joined['role_place_id'].fillna(0).astype(int)
df_joined['role_start_date_exact'] = df_joined['role_start_date_exact'].fillna(0).astype(int)
df_joined['role_classis_code'] = df_joined['role_classis_code'].fillna(0).astype(int)
df_joined['role_end_date_exact'] = df_joined['role_end_date_exact'].fillna(0).astype(int)
df_joined['role_residence_place_id'] = df_joined['role_residence_place_id'].fillna(0).astype(int)

In [11]:
# Update the table in the copied database
update_access_table(temp_database_path, '12_clerus_role', df_joined)